<a href="https://colab.research.google.com/github/TaufeeqAhmed/EIP-Session1/blob/master/Assignment_5_Cyclic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
gdrive/  hvc_annotations.csv  resized/  sample_data/


In [0]:
%tensorflow_version 1.x
!pip install bokeh
import cv2
import json
from keras.callbacks import ModelCheckpoint
from keras.callbacks import *
import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16,InceptionV3, ResNet50V2
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam,SGD,RMSprop
from keras.preprocessing.image import ImageDataGenerator
import keras
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

Using TensorFlow backend.


In [0]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True, augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.augmentation = augmentation
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [0]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15)
train_df.shape, val_df.shape

((11537, 28), (2036, 28))

In [0]:
train_df.head()

,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
11075,resized/11077.jpg,1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,0
5937,resized/5938.jpg,1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0
3339,resized/3340.jpg,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0
6745,resized/6746.jpg,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0
747,resized/748.jpg,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0


In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32)
valid_gen = PersonDataGenerator(train_df, batch_size=64, shuffle=False)

In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [0]:
backbone = ResNet50V2(
    weights=None, 
    include_top=False, 
    input_tensor=Input(shape=(224, 224, 3))
)

neck = backbone.output
neck = Flatten(name="flatten")(neck)
neck = Dense(512, activation="relu")(neck)


def build_tower(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.3)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# freeze backbone
#for layer in backbone.layers:
#	layer.trainable = False

In [0]:
# losses = {
# 	"gender_output": "binary_crossentropy",
# 	"image_quality_output": "categorical_crossentropy",
# 	"age_output": "categorical_crossentropy",
# 	"weight_output": "categorical_crossentropy",

# }
# loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0}
opt = SGD(lr=0.001, momentum=0.9)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
    # loss_weights=loss_weights, 
    metrics=["accuracy"]
)

import os
# Prepare model model saving directory.
#save_dir = os.path.join(os.getcwd(), 'saved_models')
save_dir = '/content/gdrive/My Drive/Colab Notebooks/'
model_name5 = 'weights.{epoch:02d}-{val_loss:.2f}.hdf5'

print(save_dir)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name5)

print(filepath)

#filepath="/content/drive/My Drive/saved weights/weights.{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor='val_loss', verbose=1, save_best_only=True) 
#/content/gdrive/My Drive/saved weights
#/content/gdrive/My Drive/Colab Notebooks



/content/gdrive/My Drive/Colab Notebooks/
/content/gdrive/My Drive/Colab Notebooks/weights.{epoch:02d}-{val_loss:.2f}.hdf5


In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
____________________________________________________________________________________________

In [0]:
def find_lr(model, start_lr, end_lr):
    finder = LRFinder(start_lr, end_lr, len(train_gen))
    weights = model.get_weights()    
    try:
        history = model.fit_generator(
            generator=train_gen,
            validation_data=valid_gen,
            epochs=1,
            verbose=1,
            callbacks=[finder],
        )   
    finally:    
        model.set_weights(weights)    
    return finder
    
class LRFinder(Callback):    
    def __init__(self, start_lr=1e-5, end_lr=10, step_size=None, beta=.98):
        super().__init__()
        
        self.start_lr = start_lr
        self.end_lr = end_lr
        self.step_size = step_size
        self.beta = beta
        self.lr_mult = (end_lr/start_lr)**(1/step_size)
        
    def on_train_begin(self, logs=None):
        self.best_loss = 1e9
        self.avg_loss = 0
        self.losses, self.smoothed_losses, self.lrs, self.iterations = [], [], [], []
        self.iteration = 0
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.start_lr)
        
    def on_batch_end(self, epoch, logs=None):
        logs = logs or {}
        loss = logs.get('loss')
        self.iteration += 1
        
        self.avg_loss = self.beta * self.avg_loss + (1 - self.beta) * loss
        smoothed_loss = self.avg_loss / (1 - self.beta**self.iteration)
        
        # Check if the loss is not exploding
        if self.iteration>1 and smoothed_loss > self.best_loss * 4:
            self.model.stop_training = True
            return

        if smoothed_loss < self.best_loss or self.iteration==1:
            self.best_loss = smoothed_loss
        
        lr = self.start_lr * (self.lr_mult**self.iteration)
        
        self.losses.append(loss)
        self.smoothed_losses.append(smoothed_loss)
        self.lrs.append(lr)
        self.iterations.append(self.iteration)        
        
        K.set_value(self.model.optimizer.lr, lr)  

    def plot(self, lskip=10, rskip=10):
        lrs = self.lrs[lskip:-rskip]
        losses = self.smoothed_losses[lskip:-rskip]

        output_notebook()
        p = figure(title='Learning Rate Finder', x_axis_label='LR', y_axis_label='Loss')
        p.line(lrs, losses)
        show(p)
        
        best_idxs = np.argpartition(losses, 15)[:15]
        best_lrs = np.take(lrs, best_idxs)
        print(f"Best LRs: {best_lrs}")    
#Cyclic Learning Rate
class CyclicLR(Callback):
  

    def __init__(self, base_lr=0.00000005, max_lr=0.009, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [0]:
opt = SGD()
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
      
    metrics=["accuracy"]
)
finder = find_lr(model, 0.000003, 0.00003)
finder.plot()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1
360/360 [==============================] - 113s 314ms/step - loss: 9.4595 - gender_output_loss: 0.7776 - image_quality_output_loss: 1.1471 - age_output_loss: 1.7168 - weight_output_loss: 1.2082 - bag_output_loss: 1.1675 - footwear_output_loss: 1.1746 - pose_output_loss: 1.1973 - emotion_output_loss: 1.0703 - gender_output_acc: 0.5123 - image_quality_output_acc: 0.4180 - age_output_acc: 0.2808 - weight_output_acc: 0.5114 - bag_output_acc: 0.4418 - footwear_output_acc: 0.3894 - pose_output_acc: 0.4113 - emotion_output_acc: 0.6227 - val_loss: 8.8788 - val_gender_output_loss: 0.7562 - val_image_quality_output_loss: 1.0941 - val_age_output_loss: 1.6032 - val_weight_output_loss: 1.1473 - val_bag_output_loss: 1.0445 - val_footwear_output_loss: 1.1376 - val_pose_output_loss: 1.0771 - val_emotion_output_loss: 1.0188 - val_gender_output_acc: 0.5403 - val_image_quality_output_acc: 0.4907 - va

Best LRs: [2.70817553e-05 2.74304142e-05 2.79618370e-05 2.76064233e-05
 2.60621442e-05 2.81412560e-05 2.77835618e-05 2.57308770e-05
 2.58959809e-05 2.67375281e-05 2.65670588e-05 2.69090913e-05
 2.72555273e-05 2.63976763e-05 2.55668257e-05]


In [0]:
opt = RMSprop()
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
      
    metrics=["accuracy"]
)
finder = find_lr(model, 0.000003, 0.00003)
finder.plot()

Epoch 1/1
360/360 [==============================] - 113s 314ms/step - loss: 8.2879 - gender_output_loss: 0.7185 - image_quality_output_loss: 1.0164 - age_output_loss: 1.5196 - weight_output_loss: 1.0523 - bag_output_loss: 0.9781 - footwear_output_loss: 1.0555 - pose_output_loss: 0.9887 - emotion_output_loss: 0.9587 - gender_output_acc: 0.5370 - image_quality_output_acc: 0.5165 - age_output_acc: 0.3493 - weight_output_acc: 0.6140 - bag_output_acc: 0.5172 - footwear_output_acc: 0.4711 - pose_output_acc: 0.5775 - emotion_output_acc: 0.7060 - val_loss: 7.8594 - val_gender_output_loss: 0.6819 - val_image_quality_output_loss: 0.9697 - val_age_output_loss: 1.4348 - val_weight_output_loss: 0.9856 - val_bag_output_loss: 0.9180 - val_footwear_output_loss: 0.9875 - val_pose_output_loss: 0.9331 - val_emotion_output_loss: 0.9488 - val_gender_output_acc: 0.5626 - val_image_quality_output_acc: 0.5431 - val_age_output_acc: 0.3963 - val_weight_output_acc: 0.6354 - val_bag_output_acc: 0.5627 - val_foot

Best LRs: [2.62293737e-05 1.73074213e-05 1.74184756e-05 1.75302424e-05
 2.50809208e-05 2.52418542e-05 2.58959809e-05 2.60621442e-05
 1.71970751e-05 2.55668257e-05 1.79845275e-05 2.63976763e-05
 2.47621256e-05 1.83329505e-05 2.22108766e-05]


In [0]:
# add image augmentation
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=True,
        horizontal_flip=True,
        vertical_flip=False,
    )
)
clr_triangular = CyclicLR(0.000003, 0.00003,mode='triangular')

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [18]:
history=model.fit_generator(  
    steps_per_epoch=20000 // 32,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=25,
    verbose=1,callbacks=[checkpoint,clr_triangular]
)

Epoch 1/25
  1/625 [..............................] - ETA: 20:38 - loss: 11.2477 - gender_output_loss: 0.9328 - image_quality_output_loss: 1.6790 - age_output_loss: 1.7537 - weight_output_loss: 1.5065 - bag_output_loss: 1.5793 - footwear_output_loss: 1.1042 - pose_output_loss: 1.6091 - emotion_output_loss: 1.0831 - gender_output_acc: 0.3438 - image_quality_output_acc: 0.1875 - age_output_acc: 0.2500 - weight_output_acc: 0.1875 - bag_output_acc: 0.4062 - footwear_output_acc: 0.4062 - pose_output_acc: 0.1875 - emotion_output_acc: 0.5938

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.508036). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


  2/625 [..............................] - ETA: 14:08 - loss: 10.7739 - gender_output_loss: 0.8526 - image_quality_output_loss: 1.4611 - age_output_loss: 1.7983 - weight_output_loss: 1.5406 - bag_output_loss: 1.4316 - footwear_output_loss: 1.2183 - pose_output_loss: 1.4097 - emotion_output_loss: 1.0617 - gender_output_acc: 0.4219 - image_quality_output_acc: 0.2656 - age_output_acc: 0.2500 - weight_output_acc: 0.3125 - bag_output_acc: 0.3906 - footwear_output_acc: 0.3438 - pose_output_acc: 0.2344 - emotion_output_acc: 0.6406

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.257094). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


625/625 [==============================] - 149s 238ms/step - loss: 8.1334 - gender_output_loss: 0.7098 - image_quality_output_loss: 0.9975 - age_output_loss: 1.4933 - weight_output_loss: 1.0317 - bag_output_loss: 0.9596 - footwear_output_loss: 1.0259 - pose_output_loss: 0.9692 - emotion_output_loss: 0.9464 - gender_output_acc: 0.5448 - image_quality_output_acc: 0.5274 - age_output_acc: 0.3604 - weight_output_acc: 0.6169 - bag_output_acc: 0.5284 - footwear_output_acc: 0.4932 - pose_output_acc: 0.5912 - emotion_output_acc: 0.7099 - val_loss: 7.7251 - val_gender_output_loss: 0.6700 - val_image_quality_output_loss: 0.9443 - val_age_output_loss: 1.4386 - val_weight_output_loss: 0.9817 - val_bag_output_loss: 0.9108 - val_footwear_output_loss: 0.9455 - val_pose_output_loss: 0.9058 - val_emotion_output_loss: 0.9284 - val_gender_output_acc: 0.5867 - val_image_quality_output_acc: 0.5640 - val_age_output_acc: 0.3926 - val_weight_output_acc: 0.6255 - val_bag_output_acc: 0.5625 - val_footwear_outpu

In [0]:
M1 = keras.models.load_model("/content/gdrive/My Drive/Colab Notebooks/weights.14-2.69.hdf5")

In [20]:
# add image augmentation
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=True,
        horizontal_flip=True,
        vertical_flip=False,
    )
)
clr_triangular1 = CyclicLR(0.000003, 0.00003,mode='triangular2')

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [0]:
history1=M1.fit_generator(  
    steps_per_epoch=20000 // 32,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=25,
    verbose=1,callbacks=[checkpoint,clr_triangular1]
)

Epoch 1/25
625/625 [==============================] - 153s 245ms/step - loss: 0.8441 - gender_output_loss: 0.0547 - image_quality_output_loss: 0.1057 - age_output_loss: 0.1707 - weight_output_loss: 0.1245 - bag_output_loss: 0.0916 - footwear_output_loss: 0.0953 - pose_output_loss: 0.0821 - emotion_output_loss: 0.1195 - gender_output_acc: 0.9807 - image_quality_output_acc: 0.9632 - age_output_acc: 0.9449 - weight_output_acc: 0.9576 - bag_output_acc: 0.9661 - footwear_output_acc: 0.9655 - pose_output_acc: 0.9709 - emotion_output_acc: 0.9578 - val_loss: 0.0813 - val_gender_output_loss: 0.0028 - val_image_quality_output_loss: 0.0103 - val_age_output_loss: 0.0189 - val_weight_output_loss: 0.0168 - val_bag_output_loss: 0.0073 - val_footwear_output_loss: 0.0039 - val_pose_output_loss: 0.0040 - val_emotion_output_loss: 0.0172 - val_gender_output_acc: 1.0000 - val_image_quality_output_acc: 1.0000 - val_age_output_acc: 1.0000 - val_weight_output_acc: 0.9995 - val_bag_output_acc: 1.0000 - val_foo

In [22]:
results = M1.evaluate_generator(train_gen, verbose=1)
dict(zip(M1.metrics_names, results))

360/360 [==============================] - 27s 76ms/step


{'age_output_acc': 0.9999131944444445,
 'age_output_loss': 0.02142047479800466,
 'bag_output_acc': 0.9999131944444445,
 'bag_output_loss': 0.009747387645378087,
 'emotion_output_acc': 0.9996527777777777,
 'emotion_output_loss': 0.019830955599900335,
 'footwear_output_acc': 0.9999131944444445,
 'footwear_output_loss': 0.004612071684581072,
 'gender_output_acc': 1.0,
 'gender_output_loss': 0.0031154882950229675,
 'image_quality_output_acc': 0.9999131944444445,
 'image_quality_output_loss': 0.012741984815026324,
 'loss': 0.09446600636260377,
 'pose_output_acc': 1.0,
 'pose_output_loss': 0.005379672263128062,
 'weight_output_acc': 0.9996527777777777,
 'weight_output_loss': 0.0176179710491043}

In [23]:
results = M1.evaluate_generator(valid_gen, verbose=1)
dict(zip(M1.metrics_names, results))

180/180 [==============================] - 24s 134ms/step


{'age_output_acc': 0.9999131944444445,
 'age_output_loss': 0.021380955374075306,
 'bag_output_acc': 0.9999131944444445,
 'bag_output_loss': 0.009743425019809768,
 'emotion_output_acc': 0.9996527777777777,
 'emotion_output_loss': 0.01979811869872113,
 'footwear_output_acc': 0.9999131944444445,
 'footwear_output_loss': 0.00460671701714293,
 'gender_output_acc': 1.0,
 'gender_output_loss': 0.0031151383959998687,
 'image_quality_output_acc': 0.9999131944444445,
 'image_quality_output_loss': 0.012742263559872905,
 'loss': 0.09437588718202379,
 'pose_output_acc': 1.0,
 'pose_output_loss': 0.005376545249277519,
 'weight_output_acc': 0.9996527777777777,
 'weight_output_loss': 0.01761272370867017}